In [14]:
import requests
from bs4 import BeautifulSoup
import os
import json
import random,time
from random import randint
import re
import urllib
import sys
from urllib import request
from datetime import datetime
mydir='cookpad'
mydir_save_img="cookpad_save_img"
mydir_save_log='cookpad_log'
def create_mkdir():
    if os.path.exists(mydir):
        print('mrdir name cookpad has been create')
    else:
        os.mkdir(mydir)

    if os.path.exists(mydir_save_img):
        print("mrdir name cookpad_save_img has been create")
    else:
        os.mkdir(mydir_save_img)
    
    if os.path.exists(mydir_save_log):
        print("mrdir name cookpad_save_log has been create")
    else:
        os.mkdir(mydir_save_log)

def write_to_file(img, content, cook, cook_time):
    pass


create_mkdir()
    
url="https://cookpad.com/tw"
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
ss=requests.session()

#print(title)
set_next_url_count=1
set_next_url=1
set_total_items=1
set_while_stop=1
list_save=[['食譜名稱', '作者','圖片','份數', '料理時間', '食材','食材數量', '料理步驟']]
tep_list=[]
save_next_html=[]#存下檢查爬到哪裡了
set_food=''
set_step=''
    
while (set_while_stop==1):
#for go  in  range(1,3):
    res=ss.get(url,headers=headers)
    soup=BeautifulSoup(res.text,'html.parser')
    #print(soup)    
    mainbody=soup.select('div.media')
    getnextpageurl="https://cookpad.com"+soup.select('div#feed_pagination')[0].select('a')[0]['href']
    save_next_html.append(getnextpageurl)#紀錄html
    

    for i in mainbody:
        
        print("第{}筆資料".format(set_total_items))
        title=i.select('div.media__body')[0].select('a.flex-grow')[0]['aria-label']
        test2=i.select('div.media__body')[0].select('a.flex-grow')[0]
        title_url="https://cookpad.com"+i.select('div.media__body')[0].select('a.flex-grow')[0]['href']
        url=title_url
        res=ss.get(url,headers=headers)
        soup=BeautifulSoup(res.text,'html.parser')
        #print(soup)
    
        print(title)
        #print(test2)
        print(title_url)
        time_strat=time.time()
        #time.sleep(5)
        time.sleep(randint(5,15))
        time_end=time.time()
        print(time_end-time_strat)
        #如果img沒有抓到
        img=soup.select('div.tofu_image')
        if len(img)<=0:
            print("img is not found")
            img="na"
        else:
            img=img[0].select('img')[0]['src']
        title_name=soup.select('section.intro-container')[0].select('h1.break-words')[0].text
        artice=soup.select('div.recipe-show__story')[0].select('p.mb-2')[0].text.split('#')[0]
        arthor=soup.select('section.document__section.media.author-container')[0].select('span.text-cookpad-16')[0].text
        cook_time=soup.select('div.p-3.border-t.border-quaternary.text-center.text-secondary')
        if len(cook_time) <= 0 :
            cook_time = "Na"
        else:
            cook_time = cook_time[0].select('span.px-4')[0].text #有些資料沒有
           
        food_test=soup.select('div.ingredient-list')[0].select('ol.list-unstyled.text-cookpad-18')[0].select('li')
        Number_of_copies=soup.select('div.text-secondary.mt-2')[0].text
            
        step=soup.select('ol.numbered-list')[0].select('p.mb-2')
            
           
        step_number=1
        set_food = ''
        for x in food_test:
            get_food=x.select('div')[0].text
            set_food=set_food+get_food
        
            set_step = ''
        for z in step:
            get_step=z.text
            set_step=set_step+"@"+str(step_number)+"."+get_step+"\"
            step_number=step_number+1
        step_number=1#重設步驟變數
        #print(img)
        #print(title_name)
        #print(artice)
        #print(Awards)
        #print(arthor)
        #print(cook_time)
        #print(food_total)
        #print(set_food)
        #print(set_step)
        #print(img)
        print(Number_of_copies)
        #存入資料
        title_name=title_name.replace(" ","")
        set_food=set_food.replace(" ","")
            
        tep_list.append(title_name)
        tep_list.append(artice)
        tep_list.append(arthor)
        tep_list.append(cook_time)
        tep_list.append(set_food)
        tep_list.append(set_step)
        tep_list.append(Number_of_copies)
        list_save.append(tep_list)
        
        #print(list_save)
        tep_list=[]
        # write_to_file(img, content, cook, cook_time)
           
        #print(list_save)
        #print("_____________________________________")
            
        #print("正規化檔名前")
        #print("_____________________________________")
        string=title_name
        re_title_name=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",string)
        #print(re_title_name)
        save_name_id=str(set_total_items)
        save_name=save_name_id+"_"+re_title_name
        #print(save_name)
        #print("正規化檔名後")
            
            
            
            
        #print("_____________________________________")
        #print('進入寫檔前')
     
            
            
        with open('./cookpad/'+save_name+'.txt','w',encoding="utf-8") as es:
            es.write('食譜名稱:{}'.format(title_name)+'|'+'\n')
            es.write('說明文:{}'.format(artice)+'|'+'\n')
                
            es.write('作者:{}'.format(arthor)+'|'+'\n')
            es.write('料理時間:{}'.format(cook_time)+'|'+'\n')
            es.write('所需食材:{}'.format(set_food)+'|'+'\n')
            es.write('料理步驟:{}'.format(set_step)+'|'+'\n')
            es.write('份數:{}'.format(Number_of_copies)+'|'+'\n')
            print('寫檔完成')
                
           
            
        print("_____________________________________")
            
        print("進入寫入圖片前")
            
        if img!="na":
            print("進入寫入圖片前")
            split_img=img.split(':')[1]#進行切片防止:被轉碼
            test=urllib.parse.quote(split_img)#網址中文轉碼
            save_img_url="https:"+test#進行網址重組
            request.urlretrieve(save_img_url, './cookpad_save_img/{}.jpg'.format(save_name))#寫入圖片
        else:
            print("image not found")
            
               
        print("圖片下載完成")
        set_total_items+=1

    #寫入next_URL_LOG       
    with open('./cookpad_log/'+str(set_next_url)+'.txt','w',encoding="utf-8") as es2:
        es2.write("網址{}".format(getnextpageurl)+'\n')
    set_next_url+=1
    
    url=getnextpageurl
    print("+++++++換頁+++++++++++++++++")
    print("第{}個頁面".format(set_next_url_count))
    
    print(getnextpageurl)
    set_next_url_count+=1
"""
    if set_total_items==1001:
        set_while_stop=set_while_stop-1
        print("已經有一千筆資料了")  
"""    

if getnextpageurl=='':
    set_while_stop=set_while_stop-1
    
    

print(set_total_items-1)






第1筆資料
有魚無雷電鍋蒸蛋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12828940-%E6%9C%89%E9%AD%9A%E7%84%A1%E9%9B%B7%E9%9B%BB%E9%8D%8B%E8%92%B8%E8%9B%8B
12.000989437103271

 3-4人份
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第2筆資料
蒸松阪豬 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12909440-%E8%92%B8%E6%9D%BE%E9%98%AA%E8%B1%AC
8.000275611877441

 3人份
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第3筆資料
黑糖黑芝麻拉拉餅乾條(少糖少油) link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12557861-%E9%BB%91%E7%B3%96%E9%BB%91%E8%8A%9D%E9%BA%BB%E6%8B%89%E6%8B%89%E9%A4%85%E4%B9%BE%E6%A2%9D%E5%B0%91%E7%B3%96%E5%B0%91%E6%B2%B9
14.000027894973755


寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第4筆資料
香辣蒜味雞胸蔬食義大利麵🥦 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12909334-%E9%A6%99%E8%BE%A3%E8%92%9C%E5%91%B3%E9%9B%9E%E8%83%B8%E8%94%AC%E9%A3%9F%E7%BE%A9%E5%A4%A7%E5%88%A9%E9%BA%B5%F0%9F%A5%A6
14.000096082687378

 1 份量
  
寫檔完成
_____________________________________
進

圖片下載完成
第36筆資料
麵包冰棒 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12888431-%E9%BA%B5%E5%8C%85%E5%86%B0%E6%A3%92
13.000597953796387


寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第37筆資料
芝士茄子 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12906002-%E8%8A%9D%E5%A3%AB%E8%8C%84%E5%AD%90
15.000072956085205


寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第38筆資料
氣炸~牛粒，台式馬卡龍 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/11817858-%E6%B0%A3%E7%82%B8%E7%89%9B%E7%B2%92%E5%8F%B0%E5%BC%8F%E9%A6%AC%E5%8D%A1%E9%BE%8D
13.000701189041138

 3人份
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第39筆資料
氣炸鍋-炸油豆腐 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12841318-%E6%B0%A3%E7%82%B8%E9%8D%8B-%E7%82%B8%E6%B2%B9%E8%B1%86%E8%85%90
15.000114440917969

 3人份
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第40筆資料
黯然銷魂叉燒 (超簡單!) link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12905714-%E9%BB%AF%E7%84%B6%E9%8A%B7%E9%AD%82%E5%8F%89%E7%8

8.00010633468628

 6塊
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第71筆資料
洋芹金針菇魚餅&香腸臘味糙米飯便當 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12664608-%E6%B4%8B%E8%8A%B9%E9%87%91%E9%87%9D%E8%8F%87%E9%AD%9A%E9%A4%85%E9%A6%99%E8%85%B8%E8%87%98%E5%91%B3%E7%B3%99%E7%B1%B3%E9%A3%AF%E4%BE%BF%E7%95%B6
13.000598430633545

 2人份
  
寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第72筆資料
醬油荷包蛋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12901031-%E9%86%AC%E6%B2%B9%E8%8D%B7%E5%8C%85%E8%9B%8B
11.000096797943115


寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第73筆資料
乳酸菌發酵乳蛋糕(電子鍋) link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12900900-%E4%B9%B3%E9%85%B8%E8%8F%8C%E7%99%BC%E9%85%B5%E4%B9%B3%E8%9B%8B%E7%B3%95%E9%9B%BB%E5%AD%90%E9%8D%8B
12.0002920627594


寫檔完成
_____________________________________
進入寫入圖片前
進入寫入圖片前
圖片下載完成
第74筆資料
涼拌豆捲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12670331-%E6%B6%BC%E6%8B%8C%E8%B1%86%E6%8D%B2
6.000756025314331




KeyboardInterrupt: 

In [ ]:
#print(save_next_html)
print(len(save_next_html))
print(save_next_html[47])
print(save_next_html[46])



In [ ]:
#print(list_save)
print(len(list_save))
print(list_save[990])



In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime
#包含两个不同的子列表[1,2,3,4]和[5,6,7,8]
data=DataFrame(list_save)#这时候是以行为标准写入的
print(data)

now = datetime.now()
s1 = datetime.strftime(now,'%Y-%m-%d')
s2= datetime.strftime(now,'%H:%M:%S')
s2=s2.replace(":","_")
#print (type(now))
#print (now)
print (s1)
print(s2)
save_file_data_time=s1+"_"+s2
#print(s.replace(" ","")))

data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')

In [ ]:
!pip install pandas